In [1]:
!pip install -q opendatasets
!pip install -U scikit-learn

In [43]:
import opendatasets as od
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.metrics import classification_report,confusion_matrix

In [3]:
od.download('https://www.kaggle.com/datasets/akshaypawar7/millions-of-movies')
data = pd.read_csv("/content/millions-of-movies/movies.csv")
data

Skipping, found downloaded files in "./millions-of-movies" (use force=True to force download)


,id,title,genres,original_language,overview,popularity,production_companies,release_date,budget,revenue,runtime,status,tagline,vote_average,vote_count,credits,keywords,poster_path,backdrop_path,recommendations
0,615656,Meg 2: The Trench,Action-Science Fiction-Horror,en,An exploratory dive into the deepest depths of...,8763.998,Apelles Entertainment-Warner Bros. Pictures-di...,2023-08-02,129000000.0,352056482.0,116.0,Released,Back for seconds.,7.079,1365.0,Jason Statham-Wu Jing-Shuya Sophia Cai-Sergio ...,based on novel or book-sequel-kaiju,/4m1Au3YkjqsxF8iwQy0fPYSxE0h.jpg,/qlxy8yo5bcgUw2KAmmojUKp4rHd.jpg,1006462-298618-569094-1061181-346698-1076487-6...
1,758323,The Pope's Exorcist,Horror-Mystery-Thriller,en,Father Gabriele Amorth Chief Exorcist of the V...,5953.227,Screen Gems-2.0 Entertainment-Jesus & Mary-Wor...,2023-04-05,18000000.0,65675816.0,103.0,Released,Inspired by the actual files of Father Gabriel...,7.433,545.0,Russell Crowe-Daniel Zovatto-Alex Essoe-Franco...,spain-rome italy-vatican-pope-pig-possession-c...,/9JBEPLTPSm0d1mbEcLxULjJq9Eh.jpg,/hiHGRbyTcbZoLsYYkO4QiCLYe34.jpg,713704-296271-502356-1076605-1084225-1008005-9...
2,667538,Transformers: Rise of the Beasts,Action-Adventure-Science Fiction,en,When a new threat capable of destroying the en...,5409.104,Skydance-Paramount-di Bonaventura Pictures-Bay...,2023-06-06,200000000.0,407045464.0,127.0,Released,Unite or fall.,7.340,1007.0,Anthony Ramos-Dominique Fishback-Luna Lauren V...,peru-alien-end of the world-based on cartoon-b...,/gPbM0MK8CP8A174rmUwGsADNYKD.jpg,/woJbg7ZqidhpvqFGGMRhWQNoxwa.jpg,496450-569094-298618-385687-877100-598331-4628...
3,640146,Ant-Man and the Wasp: Quantumania,Action-Adventure-Science Fiction,en,Super-Hero partners Scott Lang and Hope van Dy...,4425.387,Marvel Studios-Kevin Feige Productions,2023-02-15,200000000.0,475766228.0,125.0,Released,Witness the beginning of a new dynasty.,6.507,2811.0,Paul Rudd-Evangeline Lilly-Jonathan Majors-Kat...,hero-ant-sequel-superhero-based on comic-famil...,/qnqGbB22YJ7dSs4o6M7exTpNxPz.jpg,/m8JTwHFwX7I7JY5fPe4SjqejWag.jpg,823999-676841-868759-734048-267805-965839-1033...
4,677179,Creed III,Drama-Action,en,After dominating the boxing world Adonis Creed...,3994.342,Metro-Goldwyn-Mayer-Proximity Media-Balboa Pro...,2023-03-01,75000000.0,269000000.0,116.0,Released,You can't run from your past.,7.262,1129.0,Michael B. Jordan-Tessa Thompson-Jonathan Majo...,philadelphia pennsylvania-husband wife relatio...,/cvsXj3I9Q2iyyIo95AecSd1tad7.jpg,/5i6SjyDbDWqyun8klUuCxrlFbyw.jpg,965839-267805-943822-842942-1035806-823999-107...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
722496,314126,Die Superbullen,Crime-Comedy,de,A crime comedy directed by Otto W. Retzer.,0.600,Lisa Film,1997-01-01,0.0,0.0,89.0,Released,NaN,3.000,1.0,Wolfgang Fierek-Ottfried Fischer-Harald Krassn...,NaN,/ek3Dpv2B1MMk6ep5DeHLXC24AN6.jpg,NaN,NaN
722497,711712,Morning Musume.'17 DVD Magazine Vol.94,NaN,ja,Includes Extended Scenes of the Members from t...,0.600,NaN,2017-03-23,0.0,0.0,75.0,Released,NaN,0.000,0.0,Fukumura Mizuki-Ikuta Erina-Iikubo Haruna-Ayum...,NaN,/7UmM2Rkt9QAiFRpR1yC6ZwqPxBn.jpg,/7Nt02nfp3t6N3HNjpkUiarVB94u.jpg,NaN
722498,761747,Cries of Our Ancestors,Documentary,en,Short documentary film telling the story of th...,0.600,NaN,2020-03-03,0.0,0.0,20.0,Released,NaN,0.000,0.0,NaN,NaN,/309MLTGKoTNZCdKbW7Te1Jk3z0h.jpg,NaN,NaN
722499,857920,The Ultimate Story Of the Spitfire,NaN,en,The Supermarine Spitfire's deadly duels over S...,0.600,NaN,2004-01-01,0.0,0.0,52.0,Released,NaN,0.000,0.0,NaN,NaN,/baRetWGXcRAEEpeMv8Wz1mYN6CM.jpg,NaN,NaN


In [53]:
train = pd.read_csv("Train_data.csv")
train.columns


Index(['Unnamed: 0', 'id', 'title', 'genres', 'original_language', 'overview',
       'popularity', 'production_companies', 'release_date', 'budget',
       'revenue', 'runtime', 'status', 'tagline', 'vote_average', 'vote_count',
       'credits', 'keywords', 'poster_path', 'backdrop_path',
       'recommendations', 'result'],
      dtype='object')

In [27]:
train_data = train[["popularity","budget","revenue","runtime","vote_average","vote_count"]]
train_data["finance"]=train_data["revenue"]/train_data["budget"]
train_data["result"]=train["result"]
train_data= train_data[train_data["budget"]>0]
train_data= train_data.drop(columns=["budget","revenue"],axis=1)


<ipython-input-27-91c4d6348480>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data["finance"]=train_data["revenue"]/train_data["budget"]
<ipython-input-27-91c4d6348480>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data["result"]=train["result"]


In [28]:
X= train_data.drop("result",axis=1).values
y= train_data["result"].values
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2)
x_train

array([[2.22350000e+01, 1.34000000e+02, 7.60000000e+00, 2.38100000e+03,
        2.50457933e-01],
       [6.68800000e+00, 9.40000000e+01, 5.90000000e+00, 3.10000000e+02,
        8.34548571e-02],
       [3.72680000e+01, 1.01000000e+02, 6.90000000e+00, 3.24800000e+03,
        6.49583333e+00],
       ...,
       [1.34100000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.90909091e+00],
       [8.64700000e+00, 9.70000000e+01, 5.50000000e+00, 1.15000000e+02,
        2.36629447e+00],
       [1.43970000e+01, 9.80000000e+01, 6.11400000e+00, 4.29000000e+02,
        2.02391450e+00]])

In [47]:
KNN = KNeighborsClassifier(n_neighbors=20)
KNN.fit(x_train,y_train)
y_pred=KNN.predict(x_test)
report = classification_report(y_test,y_pred)
print(report)
matrix = confusion_matrix(y_test,y_pred)
print(matrix)

              precision    recall  f1-score   support

 Blockbuster       0.66      0.85      0.74      1043
         Hit       0.51      0.36      0.42       340
   Super Hit       0.26      0.04      0.07       167
    disaster       0.47      0.59      0.53       448
        flop       0.20      0.04      0.07       230

    accuracy                           0.58      2228
   macro avg       0.42      0.38      0.37      2228
weighted avg       0.52      0.58      0.53      2228

[[891  30  12  95  15]
 [135 121   2  76   6]
 [106  18   7  34   2]
 [119  47   2 264  16]
 [106  23   4  87  10]]


In [51]:
tree = DecisionTreeClassifier()
tree.fit(x_train,y_train)
y_pred=tree.predict(x_test)
report=classification_report(y_test,y_pred)
print(report)

              precision    recall  f1-score   support

 Blockbuster       1.00      1.00      1.00      1043
         Hit       1.00      1.00      1.00       340
   Super Hit       1.00      1.00      1.00       167
    disaster       1.00      1.00      1.00       448
        flop       1.00      1.00      1.00       230

    accuracy                           1.00      2228
   macro avg       1.00      1.00      1.00      2228
weighted avg       1.00      1.00      1.00      2228



array(['Blockbuster', 'Blockbuster', 'disaster', ..., 'flop',
       'Blockbuster', 'Blockbuster'], dtype=object)